In [ ]:
from fastapi import FastAPI  # Импортируем FastAPI для создания API
import requests  # Импортируем requests для работы с PubMed API

""" 
FastAPI — это высокопроизводительный фреймворк для создания API.
Он позволяет обрабатывать HTTP-запросы и возвращать данные в формате JSON.
Сам по себе FastAPI не является сервером — он работает через ASGI-сервер, например, Uvicorn.
"""

app = FastAPI()  # Создаём экземпляр FastAPI, который будет обрабатывать запросы

NCBI_API_KEY = "YOUR_API_KEY"  # API-ключ для доступа к PubMed

""" 
@app.get("/search_pubmed/")
Этот декоратор регистрирует эндпоинт API. Это значит, что при GET-запросе
по адресу http://127.0.0.1:8000/search_pubmed/ будет вызвана функция search_pubmed().
Параметры передаются через URL, например:
http://127.0.0.1:8000/search_pubmed/?query=CRISPR&max_results=5
"""

@app.get("/search_pubmed/")  
def search_pubmed(query: str, max_results: int = 10):  
    """
    Эта функция вызывается при GET-запросе на /search_pubmed/.

    1. Пользователь отправляет GET-запрос на URL:
       http://127.0.0.1:8000/search_pubmed/?query=CRISPR&max_results=5
    2. FastAPI передаёт параметры в функцию search_pubmed().
    3. Функция делает запрос в PubMed API, получает данные, форматирует их и отправляет клиенту.

    Параметры:
    - query (str): строка запроса для поиска в PubMed.
    - max_results (int): количество возвращаемых результатов (по умолчанию 10).

    Возвращает:
    - JSON-объект с информацией о найденных статьях.
    """

    """ 
    Формируем URL для первого запроса к PubMed API.
    Отправляем GET-запрос, чтобы получить PMIDs (идентификаторы статей).
    """
    search_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term={query}&retmode=json&retmax={max_results}&api_key={NCBI_API_KEY}"
    search_response = requests.get(search_url).json()

    """ 
    Извлекаем список PMIDs из ответа.
    Если статей не найдено, возвращаем сообщение об ошибке.
    """
    pmids = search_response.get("esearchresult", {}).get("idlist", [])
    if not pmids:
        return {"message": "No results found"}

    """ 
    Формируем URL для второго запроса в PubMed API.
    Этот запрос получает метаданные статей по их PMIDs.
    """
    pmid_str = ",".join(pmids)
    fetch_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&id={pmid_str}&retmode=json"
    fetch_response = requests.get(fetch_url).json()

    """ 
    Обрабатываем полученные данные и формируем список статей.
    """
    results = []
    for pmid in pmids:
        article = fetch_response["result"].get(pmid, {})
        results.append({
            "title": article.get("title", "No title"),
            "authors": article.get("authors", []),
            "journal": article.get("source", "Unknown"),
            "pub_date": article.get("pubdate", "Unknown"),
            "pmid": pmid,
            "url": f"https://pubmed.ncbi.nlm.nih.gov/{pmid}/"
        })

    """ 
    Отправляем готовый JSON-ответ клиенту.
    """
    return {"articles": results}
